In [27]:
# BLOQUE 1 - Carga, limpieza, agregación y relleno a nivel product_id
import pandas as pd
import numpy as np
import os   

BASE_DATOS     =r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\datasets'
INTERMEDIOS   = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\intermedios'
SALIDAS       = r'C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\output'


# Ahora carga:
productos_pred = pd.read_csv(os.path.join(BASE_DATOS, "productos_pred.txt"), sep="\t")
df             = pd.read_csv(os.path.join(BASE_DATOS, "sell-in.txt"),sep="\t")
productos      = pd.read_csv(os.path.join(BASE_DATOS, "tb_productos.txt"),   sep="\t")
stock          = pd.read_csv(os.path.join(BASE_DATOS, "tb_stocks.txt"),      sep="\t")




In [28]:
stock.head()

,periodo,product_id,stock_final
0,201810,20524,1.61267
1,201810,20311,2.93657
2,201810,20654,6.83269
3,201810,21005,1.01338
4,201810,20974,0.34595


In [29]:
## 3) Tipado y limpieza
productos_pred["product_id"] = productos_pred["product_id"].astype("category")
df["product_id"]             = df["product_id"].astype("category")
df["plan_precios_cuidados"]  = df["plan_precios_cuidados"].fillna(False).astype("bool")
df["periodo"]                = df["periodo"].astype("int32")
df["cust_request_qty"]       = df["cust_request_qty"].astype("float32")
df["cust_request_tn"]        = df["cust_request_tn"].astype("float32")
df["tn"]                     = df["tn"].astype("float32")
df["fecha"]                  = pd.to_datetime(df["periodo"].astype(str) + "01", format="%Y%m%d")

# 4) Agregación a nivel product_id–periodo
print("🔄 Agregando a nivel producto-periodo...")
sell_in_agg = (
    df
    .groupby(['periodo','product_id'], observed=True, as_index=False)
    .agg(
        tn                    = ('tn',               'sum'),
        cust_request_qty      = ('cust_request_qty', 'sum'),
        cust_request_tn       = ('cust_request_tn',  'sum'),
        plan_precios_cuidados = ('plan_precios_cuidados','max'),
        n_customers           = ('customer_id',      'nunique')
    )
)


🔄 Agregando a nivel producto-periodo...


In [31]:
# 5) Rellenado de ceros para periodos sin ventas
print("🔄 Rellenando ceros para periodos faltantes...")
periodos       = pd.period_range('2017-01', '2019-12', freq='M').strftime('%Y%m').astype(int)
productos_list = sell_in_agg['product_id'].unique()
df_univ        = pd.MultiIndex.from_product(
                         [periodos, productos_list],
                         names=['periodo','product_id']
                     ).to_frame(index=False)

sell_in_agg = (
    df_univ
    .merge(sell_in_agg, on=['periodo','product_id'], how='left')
    .fillna({
        'tn':                    0,
        'cust_request_qty':      0,
        'cust_request_tn':       0,
        'plan_precios_cuidados': False,
        'n_customers':           0
    })
)

🔄 Rellenando ceros para periodos faltantes...


In [32]:
# 6) Limpieza de catálogo y verificación
print("🔄 Limpiando y verificando catálogo de productos...")
productos_clean = (
    productos
    .drop_duplicates()
    .groupby('product_id', as_index=False)
    .agg({
        'cat1':    'first',
        'cat2':    'first',
        'cat3':    lambda x: ','.join(sorted(set(x))),
        'brand':   'first',
        'sku_size':'first'
    })
)
assert productos_clean['product_id'].is_unique, \
    "❌ Duplicados en productos_clean"
assert stock[['periodo','product_id']].drop_duplicates().shape[0] \
        == stock.shape[0], "❌ Duplicados en stock"

🔄 Limpiando y verificando catálogo de productos...


In [33]:
# 7) Merge final de tablas
print("🔄 Haciendo merge final...")
df_prod = sell_in_agg[sell_in_agg['product_id'].isin(productos_pred['product_id'])]
df_prod = df_prod.merge(productos_clean, on='product_id', how='left')
df_prod = df_prod.merge(stock,           on=['periodo','product_id'], how='left')

for col in ['cat1','cat2','cat3','brand']:
    df_prod[col] = df_prod[col].astype('category')

🔄 Haciendo merge final...


In [34]:
# 8) Creación del target
print("🔄 Creando columna 'target' (tn a dos meses vista)...")
df_prod = df_prod.sort_values(['product_id','periodo'])
df_prod['target'] = df_prod.groupby('product_id')['tn'].shift(-2)

🔄 Creando columna 'target' (tn a dos meses vista)...


In [ ]:
#10) Guardado de resultados en Drive (CSV sin compresión)
print("🔄 Guardando resultados en Drive como CSV...")
df_prod.to_csv(
    os.path.join(INTERMEDIOS, "df_limpio_product_id.csv"),
    index=False
)

print("✅ Procesamiento completado.")
print("ℹ️ Archivo CSV guardado en:", os.path.join(INTERMEDIOS, "df_limpio_product_id.csv"))

🔄 Guardando resultados en Drive como CSV...
✅ Procesamiento completado.
ℹ️ Archivo CSV guardado en: C:\Users\Elisabeth\Desktop\MAESTRIA_AUSTRAL\Labo_III\labo3-2025v\entrega_final\intermedios\df_limpio_product_id.csv
